# Prediction

----------------------
This notebook allows for predictions on large images. It includes a comparison with the baseline model.

In [ ]:
# Imports
from patchify import patchify, unpatchify
import cv2
import numpy as np
from matplotlib import pyplot as plt
from keras.models import load_model
import os
import sys

sys.path.append('../modeling')
from predict import make_pred, single_image_IoU, map_func

### Loading and preparing the data

In [ ]:
#Load model
path = '../models/'
model_name = 'StdUnet_diceplusfocal_epochs100_batchsize8_learningrate0.0003.hdf5'
model = load_model(path+model_name, compile=False)

In [ ]:
#Load all test images and masks
test_img_dir = '../data/data_original/test_data/image/'
test_msk_dir = '../data/data_original/test_data/mask/'

img_list = os.listdir(test_img_dir)
img_list.sort()
msk_list = os.listdir(test_msk_dir)
msk_list.sort()

#Capture training image info as a list
test_images = []
for i in range(len(img_list)):
    image_name = test_img_dir + img_list[i]
    img = cv2.imread(image_name, 0)       
    test_images.append(img)

#Capture mask/label info as a list
test_masks = []
for i in range(len(msk_list)):
    image_name = test_msk_dir + msk_list[i]
    img = cv2.imread(image_name, 0)       
    test_masks.append(img)  

In [ ]:
# One-hot encode the test_masks
test_masks_oh = []
oh_dict = {28:1, 124:2, 222:3}

for i in range(len(test_masks)):
    single_mask = test_masks[i]
    single_mask = np.array(single_mask)
    # vectorize and run map_func
    vfunc  = np.vectorize(map_func)
    test_masks_oh.append(vfunc(single_mask, oh_dict))

### Predict on images

In [ ]:
# Instantiate patches list
all_image_patches = []
all_mask_patches = []
predictions = []

# Instantiate image wise IoU lists
iw_mean_IoU = []
c1_mean_IoU = []
c2_mean_IoU = []
c3_mean_IoU = []
c4_mean_IoU = []

for i in range(len(test_images)):
    # Load single image
    print(f'--------Calc. image nr.{i+1}--------')
    image = test_images[i]
    mask = test_masks_oh[i]

    # Creates patches
    patches = patchify(image, (512, 512), step=512) 
    all_image_patches.append(patches)
    patches_mask = patchify(mask, (512, 512), step=512)
    all_mask_patches.append(patches_mask)

    # Make predictions and save predictions: Predict and flatten all 512*512 patches of one image and one mask
    print('creating patches...')
    predicted_patches, mask_patches, pred_patches, true_patches = make_pred(patches, patches_mask, model)
    predictions.append(predicted_patches)
    print('finished creating patches')

    # Unpatchifying images and plotting them
    predicted_patches_reshaped = np.reshape(predicted_patches, (6, 8, 512,512)) 
    reconstructed_predictions = unpatchify(predicted_patches_reshaped, (3072, 4096))

    fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True,
                            figsize=(40,40))
    ax0.set_title('Image')
    ax0.imshow(image, cmap='gray')
    ax1.set_title('True Mask')
    ax1.imshow(mask, cmap = 'gray')
    ax2.set_title('Predicted Mask')
    ax2.imshow(reconstructed_predictions, cmap = 'gray')
    plt.show()

    #Calculating class IoUs for a single image and append
    print('calculating IoUs...')  
    mean_IoU, class_1_IoU, class_2_IoU, class_3_IoU, class_4_IoU = single_image_IoU(true_patches, pred_patches)
    iw_mean_IoU.append(round(sum(mean_IoU)/len(mean_IoU), 2))  
    c1_mean_IoU.append(round(sum(class_1_IoU)/len(class_1_IoU), 2))
    c2_mean_IoU.append(round(sum(class_2_IoU)/len(class_2_IoU), 2))
    c3_mean_IoU.append(round(sum(class_3_IoU)/len(class_3_IoU), 2))
    c4_mean_IoU.append(round(sum(class_4_IoU)/len(class_4_IoU), 2))
    print('finished calculating IoUs')

### Plot of IoU and comparison with baseline model

#### Create IoU's for the baseline model (comparison between autocreated masks and corrected masks)

In [ ]:
# Mean IoU's over all predicted images
iw_mean_IoU_model = round(sum(iw_mean_IoU)/len(iw_mean_IoU), 2)
c1_mean_IoU_model = round(sum(c1_mean_IoU)/len(c1_mean_IoU), 2)
c2_mean_IoU_model = round(sum(c2_mean_IoU)/len(c2_mean_IoU), 2)
c3_mean_IoU_model = round(sum(c3_mean_IoU)/len(c3_mean_IoU), 2)
c4_mean_IoU_model = round(sum(c4_mean_IoU)/len(c4_mean_IoU), 2)

In [ ]:
# Plots
# set width of bars
barWidth = 0.25
 
# baseline model numbers derived from EDA-and-modeling.ipynb
baseline_model = [0.99, 1.0, 0.76, 0.92, 0.99]
trained_model = [iw_mean_IoU_model, c1_mean_IoU_model, c2_mean_IoU_model, c3_mean_IoU_model, c4_mean_IoU_model]
 
# Set position of bar on X axis
r1 = np.arange(len(baseline_model))
 
# Make the plot
plt.figure(figsize=(18,10))
plt.bar(r1-0.125, baseline_model, color='gray', width=barWidth, edgecolor='black', label='Baseline model')
plt.bar(r1+0.125, trained_model, color='teal', width=barWidth, edgecolor='black', label='Trained model')
 
# Add xticks on the middle of the group bars
plt.xlabel('Type', fontweight='bold')
plt.xticks(r1, ['Average', 'Background', 'Fractures', 'Pores', 'Tiny Pores'])
plt.ylabel('IoU', fontweight='bold')
plt.savefig(f'../models/{model_name}.jpg', dpi=150)
 
# Create legend & Show graphic
plt.legend()
plt.show()